In [1]:
import geopandas as gpd
from geodatasets import get_path

# GeoTiff Images
import rasterio
from rasterio.features import geometry_mask
from rasterio.mask import mask
from osgeo import gdal
from shapely.geometry import Polygon, mapping

# Visualisation
from matplotlib import pyplot as plt
import matplotlib.image as img
from matplotlib.pyplot import figure
from PIL import Image

# Model Building
import ultralytics
from ultralytics import YOLO
import labelme2yolo
import numpy as np

# Others
import os
import shutil
import zipfile

In [2]:
# Read the shapefile
# shp_file = get_path("building_blueprint")
shp_file = r"D:\Development\git_projects\Geospatial_CV\building_blueprint\building_footprint_roi_challenge.shp"
gdf = gpd.read_file(shp_file)


In [3]:
gdf

,latitude,longitude,area_in_me,confidence,full_plus_,geometry
0,18.448569,-66.146916,None,0.7140,77CMCVX3+C6GP,"POLYGON ((167578.511 2042703.134, 167575.734 2..."
1,18.411424,-66.182499,None,0.8935,77CMCR69+H26R,"POLYGON ((163749.148 2038647.349, 163737.189 2..."
2,18.429443,-66.167636,None,0.8074,77CMCRHJ+QWHP,"POLYGON ((165354.588 2040621.133, 165350.801 2..."
3,18.371113,-66.119793,None,0.8103,77CM9VCJ+C3WG,"POLYGON ((170294.617 2034071.175, 170289.741 2..."
4,18.418506,-66.172524,None,0.8449,77CMCR9G+CX56,"POLYGON ((164821.285 2039417.821, 164806.291 2..."
...,...,...,...,...,...,...
281567,18.438485,-66.119304,None,0.7072,77CMCVQJ+97WF,"POLYGON ((170479.843 2041537.071, 170474.647 2..."
281568,18.417240,-66.164903,None,0.6713,77CMCR8P+V2XC,"POLYGON ((165618.117 2039265.732, 165611.185 2..."
281569,18.396331,-66.119886,None,0.7770,77CM9VWJ+G2Q8,"POLYGON ((170344.030 2036873.416, 170329.004 2..."
281570,18.390318,-66.178915,None,0.8523,77CM9RRC+4CGP,"POLYGON ((164091.330 2036312.981, 164081.023 2..."


In [7]:
# Define your latitude, longitude
lat, lon, poly = gdf.head(1)['latitude'].values[0], gdf.head(1)['longitude'].values[0],gdf['geometry'].head(1).values
print(lat, lon, poly)

18.448569 -66.1469158 <GeometryArray>
[<POLYGON ((167578.511 2042703.134, 167575.734 2042703.146, 167575.756 204270...>]
Length: 1, dtype: geometry


In [18]:
with rasterio.open('D:\Development\git_projects\Geospatial_CV\Pre_Event_San_Juan.tif') as src:
    # Transform polygon to raster CRS
    poly_ras = gpd.GeoSeries(poly, crs=gdf.crs).to_crs(src.crs)[0]
    
    # Get the geometry in the format rasterio wants
    geom = [mapping(poly_ras)]
    
    # Create a mask with the polygon
    mask = geometry_mask([geom for geom in geom],
                         transform=src.transform,
                         out_shape=src.shape,
                         invert=True)
    out_image = src.read(1, masked=True)
    out_image.mask = mask

# Convert the image array to a PIL Image object
im = Image.fromarray(np.uint8(out_image))

plt.imshow(im)
plt.show()